In [11]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import pyarrow as pa
import pyarrow.dataset as ds

In [22]:
config = {
    'num_rows': 2000,
    'image_size': 64
}

In [24]:
# Load the huggingface dataset
dataset = load_dataset('jonathan-roberts1/EuroSAT')

In [ ]:
#select subset of data and shuffle 
dataset = dataset['train'].shuffle(seed=42).select(range(config['num_rows']))

In [ ]:
#Split into train and test+validation
train_test_valid = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_valid['train']
test_valid_dataset = train_test_valid['test']

#Split the test+validation set into test and validation
test_valid_split = test_valid_dataset.train_test_split(test_size=0.5, seed=42)
test_dataset = test_valid_split['test']
validation_dataset = test_valid_split['train']

# Check sizes of the splits
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(validation_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Training set size: 1600
Validation set size: 200
Test set size: 200


In [ ]:
#create dataset class
class EuroSATDataset(Dataset):
    def __init__(self, hf_dataset, transform = None):
        self.data = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        image = torch.tensor(item['image'], dtype=torch.float32)
        label = torch.tensor(item['label'], dtype=torch.long)

        if self.transform:
            image = self.transform(image)

        return image, label

data_transform = transforms.Compose([transforms.Resize((config['image_size'], config['image_size']))])
    

In [ ]:
train_dataset = EuroSATDataset(train_data, data_transform)
val_dataset = EuroSATDataset(val_data, data_transform)

In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle = True)

In [7]:
samples, labels = next(iter(train_dataloader))


KeyError: 'image'

In [9]:
print(dataset['train'].column_names)

['_data_files', '_fingerprint', '_format_columns', '_format_kwargs', '_format_type', '_indexes', '_output_all_columns', '_split']
